# imports

In [1]:
import torch
torch.cuda.set_device(1)
if torch.cuda.is_available():
    current_gpu = torch.cuda.current_device()
    print(f"Current default GPU index: {current_gpu}")
    print(f"Current default GPU name: {torch.cuda.get_device_name(current_gpu)}")
else:
    print("No GPUs available.")

Current default GPU index: 1
Current default GPU name: NVIDIA A40


In [2]:
# all imports

# buffer
from collections import defaultdict
import random
from abc import ABC, abstractmethod
from typing import (
    Any,
    Dict,
    Generic,
    Optional,
    List,
    TYPE_CHECKING,
    Set,
    TypeVar,
)

import torch
from numpy import inf
from torch import cat, Tensor
from torch.nn import Module
from torch.utils.data import DataLoader

from avalanche.benchmarks.utils import (
    classification_subset,
    AvalancheDataset,
)
from avalanche.models import FeatureExtractorBackbone
# from ..benchmarks.utils.utils import concat_datasets
from avalanche.benchmarks.utils import concat_datasets
from avalanche.training.storage_policy import ReservoirSamplingBuffer, BalancedExemplarsBuffer, ClassBalancedBuffer
# from avalanche.training.storage_policy
if TYPE_CHECKING:
    from .templates import SupervisedTemplate, BaseSGDTemplate

from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins import SupervisedPlugin
from typing import Optional, TYPE_CHECKING

from avalanche.benchmarks.utils import concat_classification_datasets
from avalanche.training.plugins.strategy_plugin import SupervisedPlugin
from avalanche.training.storage_policy import (
    ExemplarsBuffer,
    ExperienceBalancedBuffer,
)

if TYPE_CHECKING:
    from avalanche.training.templates import SupervisedTemplate


# dataset
from avalanche.benchmarks import SplitMNIST, SplitCIFAR100
from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader, ReplayDataLoader

from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset


import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

import torch.optim.lr_scheduler # ?

from avalanche.benchmarks.classic import SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR10

from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark

from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark, \
                                            tensors_benchmark, paths_benchmark

from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop
from torchvision.transforms.functional import center_crop

from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip, Resize
import os

from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.logging import InteractiveLogger, TensorboardLogger, \
    WandBLogger, TextLogger

from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics

from avalanche.training.plugins.checkpoint import CheckpointPlugin, \
    FileSystemCheckpointStorage
from avalanche.training.determinism.rng_manager import RNGManager

# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

from torchvision.models import resnet18, ResNet18_Weights

# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.training.plugins import ReplayPlugin

from types import SimpleNamespace

from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy


# Replay Plugin

In [3]:
# from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
# from avalanche.training.plugins import SupervisedPlugin

# class CustomReplay(SupervisedPlugin):
#     def __init__(
#         self,
#         mem_size: int = 200,
#         batch_size: Optional[int] = None,
#         batch_size_mem: Optional[int] = None,
#         task_balanced_dataloader: bool = False,
#         storage_policy: Optional["ExemplarsBuffer"] = None,
#         # The policy that controls how to add new exemplars in memory
#                         #
#     ):
#         super().__init__()
#         self.mem_size = mem_size
#         self.batch_size = batch_size
#         self.batch_size_mem = batch_size_mem
#         self.task_balanced_dataloader = task_balanced_dataloader

#         self.storage_policy = storage_policy
#         assert storage_policy.max_size == self.mem_size

#     def before_training_exp(self,
#                             strategy: "SupervisedTemplate",
#                             num_workers: int = 0,
#                             shuffle: bool = True,
#                             **kwargs):
#         """ Here we set the dataloader. create batch with examples from both
#         training data and external memory"""
#         if len(self.storage_policy.buffer) == 0:
#             # first experience. We don't use the buffer, no need to change
#             # the dataloader.
#             return

#         # replay dataloader samples mini-batches from the memory and current
#         # data separately and combines them together.
#         print("Override the dataloader.")
#         strategy.dataloader = ReplayDataLoader(
#             strategy.adapted_dataset,
#             self.storage_policy.buffer,
#             oversample_small_tasks=True,
#             num_workers=num_workers,
#             batch_size=strategy.train_mb_size,
#             shuffle=shuffle)

#     def after_training_exp(self, strategy: "SupervisedTemplate", **kwargs):
#         """ We update the buffer after the experience.
#             You can use a different callback to update the buffer in a different place
#         """
#         print("Buffer update.")
#         self.storage_policy.update(strategy, **kwargs)

from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins import SupervisedPlugin
from typing import Optional, TYPE_CHECKING

from avalanche.benchmarks.utils import concat_classification_datasets
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins.strategy_plugin import SupervisedPlugin
from avalanche.training.storage_policy import (
    ExemplarsBuffer,
    ExperienceBalancedBuffer,
)

if TYPE_CHECKING:
    from avalanche.training.templates import SupervisedTemplate

import random
def sample_Avadataset(dataset, percentage= 0.1):
    """random sample in certain percentage, percentage should be float"""
#     random.seed(41)
    torch.manual_seed(41)
    data_len = len(dataset)
#     indices = list(range(data_len))
    indices = torch.randperm(data_len).tolist()
#     random.shuffle(indices)
    sample_num = int(data_len * percentage)
#     sampled_indices = random.sample(indices, sample_num)
#     new_buffer_data = dataset.subset(sampled_indices)
    sampled_indices = indices[:sample_num]  # Select the first 'sample_num' indices
    new_buffer_data = dataset.subset(sampled_indices)

    return new_buffer_data



class CustomReplay(SupervisedPlugin):
    def __init__(
        self,
        mem_size: int = 200,
        batch_size: Optional[int] = None,
        batch_size_mem: Optional[int] = None,
        task_balanced_dataloader: bool = False,
        storage_policy: Optional["ExemplarsBuffer"] = None,
        # The policy that controls how to add new exemplars in memory
                        #
    ):
        super().__init__()
        self.mem_size = mem_size
        self.batch_size = batch_size
        self.batch_size_mem = batch_size_mem
        self.task_balanced_dataloader = task_balanced_dataloader

        self.storage_policy = storage_policy
        assert storage_policy.max_size == self.mem_size



    def before_training_exp(self,
                            strategy: "SupervisedTemplate",
                            num_workers: int = 0,
                            shuffle: bool = True,
                            **kwargs):
        """ Here we set the dataloader. create batch with examples from both
        training data and external memory"""
        if len(self.storage_policy.buffer) == 0:
            return

        # replay dataloader samples mini-batches from the memory and current
        # data separately and combines them together.
        print("Override the dataloader.")

        buffer_size = len(self.storage_policy.buffer)
        print("buffer size: " + str(buffer_size))
        num_class = len(self.storage_policy.buffer_datasets)
        print("current class number in replay buffer: " + str(num_class))
        # for item in self.storage_policy.buffer:
        # tempory_buffer = AvalancheDataset([])

        tempory_buffer = []

        for i, dataset in enumerate(self.storage_policy.buffer_datasets):
            sub_data = sample_Avadataset(dataset)
            if i == 0:
                tempory_buffer = sub_data
            else:
                tempory_buffer = tempory_buffer.concat(sub_data)


        # buffer = [sample_Avadataset(dataset) for dataset in self.storage_policy.buffer_datasets]
        strategy.dataloader = ReplayDataLoader(
            strategy.adapted_dataset,
            # self.storage_policy.buffer,
            tempory_buffer,
            num_workers=num_workers,
            batch_size=strategy.train_mb_size,
            shuffle=shuffle)
        # for x, y, t in dl:
        #     print(t.tolist())
        #     print(len(t.tolist()))

    def after_training_exp(self, strategy: "SupervisedTemplate", **kwargs):
        """ We update the buffer after the experience.
            You can use a different callback to update the buffer in a different place
        """
        print("Buffer update.")
        self.storage_policy.update(strategy, **kwargs)


In [4]:
# self.storage_policy.update(strategy, **kwargs)-> parametric buffer update

In [5]:
from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset

class NewParametricBuffer(BalancedExemplarsBuffer):
    """Stores samples for replay using a custom selection strategy and
    grouping."""

    def __init__(
        self,
        max_size: int,
        groupby=None,
        selection_strategy: Optional["ExemplarsSelectionStrategy"] = None,
    ):
        """Init.
        :param max_size: The max capacity of the replay memory.
        :param groupby: Grouping mechanism. One of {None, 'class', 'task',
            'experience'}.
        :param selection_strategy: The strategy used to select exemplars to
            keep in memory when cutting it off.
        """
        super().__init__(max_size)
        assert groupby in {None, "task", "class", "experience"}, (
            "Unknown grouping scheme. Must be one of {None, 'task', "
            "'class', 'experience'}"
        )
        self.groupby = groupby
        ss = selection_strategy or RandomExemplarsSelectionStrategy()
        self.selection_strategy = ss
        self.seen_groups: Set[int] = set()
        self._curr_strategy = None


    def update(self, strategy: "SupervisedTemplate", **kwargs):
        assert strategy.experience is not None
        new_data: AvalancheDataset = strategy.experience.dataset
        new_groups = self._make_groups(strategy, new_data)
        self.seen_groups.update(new_groups.keys())

        # associate lengths to classes
        lens = self.get_group_lengths(len(self.seen_groups))
        group_to_len = {}
        for group_id, ll in zip(self.seen_groups, lens):
            group_to_len[group_id] = ll

        # update buffers with new data
        for group_id, new_data_g in new_groups.items():
            ll = group_to_len[group_id]
            if group_id in self.buffer_groups:
                old_buffer_g = self.buffer_groups[group_id]
                old_buffer_g.update_from_dataset(strategy, new_data_g)
                old_buffer_g.resize(strategy, ll)
            else:
                new_buffer = _ParametricSingleBuffer(ll, self.selection_strategy)
                new_buffer.update_from_dataset(strategy, new_data_g)
                self.buffer_groups[group_id] = new_buffer

        # resize buffers
        for group_id, class_buf in self.buffer_groups.items():
            self.buffer_groups[group_id].resize(strategy, group_to_len[group_id])

    def _make_groups(
        self, strategy, data: AvalancheDataset
    ) -> Dict[int, AvalancheDataset]:
        """Split the data by group according to `self.groupby`."""
        # if self.groupby is None:
        #     return {0: data}
        # elif self.groupby == "task":
        #     return self._split_by_task(data)
        # elif self.groupby == "experience":
        #     return self._split_by_experience(strategy, data)
        # elif self.groupby == "class":
        return self._split_by_class(data)
        # else:
        #     assert False, "Invalid groupby key. Should never get here."

    def _split_by_class(self, data: AvalancheDataset) -> Dict[int, AvalancheDataset]:
        # Get sample idxs per class
        cl_idxs: Dict[int, List[int]] = defaultdict(list)
        targets = getattr(data, "targets")
        for idx, target in enumerate(targets):
            target = int(target)
            cl_idxs[target].append(idx)

        # Make AvalancheSubset per class
        new_groups: Dict[int, AvalancheDataset] = {}
        for c, c_idxs in cl_idxs.items():
            new_groups[c] = classification_subset(data, indices=c_idxs)
        return new_groups

    # def _split_by_experience(
    #     self, strategy, data: AvalancheDataset
    # ) -> Dict[int, AvalancheDataset]:
    #     exp_id = strategy.clock.train_exp_counter + 1
    #     return {exp_id: data}

    # def _split_by_task(self, data: AvalancheDataset) -> Dict[int, AvalancheDataset]:
    #     new_groups = {}
    #     task_set = getattr(data, "task_set")
    #     for task_id in task_set:
    #         new_groups[task_id] = task_set[task_id]
    #     return new_groups



class _ParametricSingleBuffer(ExemplarsBuffer):
    """A buffer that stores samples for replay using a custom selection
    strategy.

    This is a private class. Use `ParametricBalancedBuffer` with
    `groupby=None` to get the same behavior.
    """

    def __init__(
        self,
        max_size: int,
        selection_strategy: Optional["ExemplarsSelectionStrategy"] = None,
    ):
        """
        :param max_size: The max capacity of the replay memory.
        :param selection_strategy: The strategy used to select exemplars to
                                   keep in memory when cutting it off.
        """
        super().__init__(max_size)
        ss = selection_strategy or RandomExemplarsSelectionStrategy()
        self.selection_strategy = ss
        self._curr_strategy = None

    def update(self, strategy: "SupervisedTemplate", **kwargs):
        assert strategy.experience is not None
        new_data = strategy.experience.dataset
        self.update_from_dataset(strategy, new_data)

    def update_from_dataset(self, strategy, new_data):
        # print(new_data[1])
        data_len = len(new_data)
        indices = list(range(data_len))
        random.shuffle(indices)
        sample_num = int(data_len * 0.2)
        sampled_indices = random.sample(indices, sample_num)
        # new_buffer_data = new_data[:sample_num]
        # new_buffer_data = Subset(new_data, sampled_indices)
        new_buffer_data = new_data.subset(sampled_indices)

        self.buffer = self.buffer.concat(new_buffer_data)
        self.resize(strategy, self.max_size)

    def resize(self, strategy, new_size: int):
        self.max_size = new_size
        idxs = self.selection_strategy.make_sorted_indices(
            strategy=strategy, data=self.buffer
        )
        self.buffer = self.buffer.subset(idxs[: self.max_size])


In [6]:
# def data
#     total_indices = len(data)
#     num_samples = int(total_indices * self.percentage)  # Calculate number of samples based on percentage

#     indices = list(range(total_indices))  # Generate a list of all indices
#     random.shuffle(indices)

#     sampled_indices = random.sample(indices, num_samples)  # Randomly sample indices

#     return sampled_indices

In [7]:
# class ParametricBuffer(BalancedExemplarsBuffer):
#     def __init__(self, max_size: int, percent_samples_per_class: float, groupby='class', selection_strategy: Optional["ExemplarsSelectionStrategy"] = None):
#         """
#         :param percent_samples_per_class: The percentage of samples to keep from each class after an update.
#         """
#         super().__init__(max_size, groupby, selection_strategy)
#         self.percent_samples_per_class = percent_samples_per_class

#     def update(self, strategy: "SupervisedTemplate", **kwargs):
#         super().update(strategy, **kwargs)  # Assuming super().update does the necessary updates
#         self._adjust_buffer_to_percent()

#     def _adjust_buffer_to_percent(self):
#         """
#         Adjusts the buffer to retain only a specified percentage of examples from each class.
#         """
#         for group_id, buffer_group in self.buffer_groups.items():
#             num_samples_to_keep = int(len(buffer_group) * self.percent_samples_per_class)
#             if len(buffer_group) > num_samples_to_keep:
#                 selected_indices = np.random.choice(len(buffer_group), num_samples_to_keep, replace=False)
#                 self.buffer_groups[group_id] = buffer_group.subset(selected_indices)


In [8]:
l = [1,2,3,4]
random.sample(l, 2)

[4, 2]

# Data transform

In [9]:
# !pip install pytorch_pretrained_vit


import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

import torch.optim.lr_scheduler # ?

from avalanche.benchmarks.classic import SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR10

from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark

from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark, \
                                            tensors_benchmark, paths_benchmark

from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop
from torchvision.transforms.functional import center_crop

In [10]:
from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip, Resize
import os
# stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
# stats = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

transform_train = Compose([
    Resize(224),
    # Resize(384),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize(*stats,inplace=True)
])

transform_test = Compose([
    Resize(224),
    # Resize(384),
    ToTensor(),
    Normalize(*stats,inplace=True)
])

# from torchvision.transforms import Resize
# transform1 = Compose([
#     Resize(224),
#     ToTensor()
# ])

In [11]:
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.logging import InteractiveLogger, TensorboardLogger, \
    WandBLogger, TextLogger
# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('logs/method2_replay_real_resample_dino_50epochs.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

In [12]:
# evaluation
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics

# The evaluation plugin manages the metrics computation.
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    # forgetting_metrics(experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger])

In [13]:
from avalanche.training.plugins.checkpoint import CheckpointPlugin, \
    FileSystemCheckpointStorage
from avalanche.training.determinism.rng_manager import RNGManager


In [14]:
RNGManager.set_random_seeds(1234)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

checkpoint_plugin = CheckpointPlugin(
    FileSystemCheckpointStorage(
        directory='./checkpoints/task_cifar',
    ),
    map_location=device
)

# Load checkpoint (if exists in the given storage)
# If it does not exist, strategy will be None and initial_exp will be 0
strategy, initial_exp = checkpoint_plugin.load_checkpoint_if_exists()

cuda


In [15]:
# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

In [16]:
from torchvision.models import resnet18, ResNet18_Weights
# weights = ResNet18_Weights.DEFAULT
# resnet_model = resnet18(weights=weights)

In [17]:
import torch
import torch.nn as nn
# resnet50 = torch.hub.load('ckpt/dino_resnet50_pretrain.pth', 'dino_resnet50')
resnet_model = torch.hub.load('facebookresearch/dino:main', 'dino_resnet50')
resnet_model.fc = nn.Identity()

for param in resnet_model.parameters():
    param.requires_grad = False
    

# Step 2: Create a new network class with an additional linear layer
class CustomNetwork(nn.Module):
    def __init__(self, pretrained_model, num_classes):
        super(CustomNetwork, self).__init__()
        self.pretrained_model = pretrained_model
        self.fc = nn.Linear(2048, num_classes)  # New linear layer with trainable weights

    def forward(self, x):
        # Extract features using the pre-trained model
        features = self.pretrained_model(x)
        # Flatten the output for the linear layer if not already flattened
        features = torch.flatten(features, 1)
        # Pass features through the new linear layer
        output = self.fc(features)
        return output

# Step 3: Initialize the new network with the desired number of output classes
num_classes = 100  # Example: 10 classes for a new classification task
resnet_model = CustomNetwork(resnet_model, num_classes)

# The only parameters that are trainable are those of the new linear layer
trainable_params = sum(p.numel() for p in resnet_model.parameters() if p.requires_grad)
print(f'Trainable parameters: {trainable_params}')  # Should print > 0 if the fc layer is correctly set


Using cache found in /homes/55/enbo/.cache/torch/hub/facebookresearch_dino_main
/homes/55/enbo/miniconda3/envs/thesis3.7/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/homes/55/enbo/miniconda3/envs/thesis3.7/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Trainable parameters: 204900


In [18]:
# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.training.plugins import ReplayPlugin

from types import SimpleNamespace

In [19]:
benchmark = SplitCIFAR100(n_experiences=20,
                          train_transform=transform_train,
                          eval_transform = transform_test,
                          seed = 41
                          )

Files already downloaded and verified
Files already downloaded and verified


In [20]:
from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy
from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy
storage_p = ParametricBuffer(
    max_size=60000,
    groupby='class',
    selection_strategy=RandomExemplarsSelectionStrategy()
    # selection_strategy=RandomExemplarsSelectionStrategy()

)


print(f"Max buffer size: {storage_p.max_size}, current size: {len(storage_p.buffer)}")
for i in range(5):
    strategy_state = SimpleNamespace(experience=benchmark.train_stream[i])
    # print(len(benchmark.train_stream[i]))
    storage_p.update(strategy_state)
    print(f"Max buffer size: {storage_p.max_size}, current size: {len(storage_p.buffer)}")
    # print(f"class targets: {storage_p.buffer.targets}\n")

Max buffer size: 60000, current size: 0
Max buffer size: 60000, current size: 2500
Max buffer size: 60000, current size: 5000
Max buffer size: 60000, current size: 7500
Max buffer size: 60000, current size: 10000
Max buffer size: 60000, current size: 12500


## change epochs

In [21]:
storage_p = ParametricBuffer(
    max_size=60000,
    groupby='class',
    selection_strategy=RandomExemplarsSelectionStrategy()
    # selection_strategy=RandomExemplarsSelectionStrategy()

)


In [22]:
cl_strategy = Naive(
    resnet_model, torch.optim.SGD(resnet_model.fc.parameters(), lr=0.01, momentum = 0.9),
    CrossEntropyLoss(), train_mb_size=32, train_epochs=50, eval_mb_size=16,
    # eval_every=500,
    device=device,
    evaluator=eval_plugin,
    plugins=[CustomReplay(mem_size=60000, storage_policy = storage_p)]
    )
# mem_size: int = 200,
        # batch_size: Optional[int] = None,
        # batch_size_mem: Optional[int] = None,
        # task_balanced_dataloader: bool = False,
        # storage_policy: Optional["ExemplarsBuffer"] = None,

In [ ]:
# Training
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 36, 5, 20, 54]
-- >> Start of training phase << --
100%|██████████████████████████████████████████████████████████████████████████████| 79/79 [00:08<00:00,  9.54it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.4102
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6640
100%|██████████████████████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 15.00it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.6029
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8476
100%|██████████████████████████████████████████████████████████████████████████████| 79/79 [00:05<00:00, 15.39it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4572
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8868
100%|██████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 16.33it/s]
Epoch 3 ended.
	Loss_Epoch/train_ph

100%|██████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 17.20it/s]
Epoch 33 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1265
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9692
100%|██████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 17.69it/s]
Epoch 34 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1292
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9688
100%|██████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 17.88it/s]
Epoch 35 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1309
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9680
100%|██████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 17.72it/s]
Epoch 36 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1262
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9672
100%|███████████████████

100%|██████████████████████████████████████████████████████████████████████████████| 32/32 [00:01<00:00, 28.06it/s]
> Eval on experience 11 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp011 = 12.2923
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp011 = 0.0000
-- Starting eval on experience 12 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████| 32/32 [00:01<00:00, 28.65it/s]
> Eval on experience 12 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp012 = 12.8392
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp012 = 0.0000
-- Starting eval on experience 13 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████| 32/32 [00:01<00:00, 30.07it/s]
> Eval on experience 13 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp013 = 12.9077
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp013 = 0.000

In [ ]:
results